# Welcome

Stefano Camborda La Cruz
  
  
*Metabolic System Interactions*

GOAL: Use a generic plant metabolic model to create a C4 leaf model

Basic concepts and ideas:
- Extending a large-scale metabolic model
- Reproducibility and robustness
- Implementation using COBRApy and CobraMod

*duration: approx. 1h*

### C4 photosynthesis in a nutshell

NADINE: could explain briefly here

| <img src="https://miro.medium.com/max/2400/0*BhzQZYPEV8cGj5mZ.png" alt="drawing" width="680"/> | <img src="https://miro.medium.com/proxy/1*duJX24NyLJ5osWdBHU8Kxg.png" alt="drawing" width="680"/> |
|------------------------------------------------------------------------------------------------|---------------------------------------------------------------------------------------------------|

## Creating the C4 leaf model

Load basic Python modules

In [1]:
from pathlib import Path

from cobra.io import read_sbml_model
from cobra import Solution, Model

from cobramod import add_reactions

Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled


/home/stefano/miniconda3/envs/test/lib/python3.9/site-packages/cobramod/parsing/biocyc.py:66: UserWarning: Biocyc version is not the same in CobraMod. Please inform maintainers.
  warn(


### Preparation

Loading the model and defining directories

In [2]:
# Load main model
backup = read_sbml_model(
    str(Path.cwd().joinpath("model", "PlantCoreMetabolism_v2_0_0.sbml"))
)
dir_data = Path.cwd().joinpath("data")

model = backup.copy()

assert backup
assert dir_data.exists()

In [3]:
# Skipped
# Fix compartment metabolites
for metabolite in [
    "FERULIC_ACID_p", "VANILLIN_p", "FERULOYL_COA_p"
]:
    model.metabolites.get_by_id(metabolite).compartment = "p"
    assert model.metabolites.get_by_id(metabolite).compartment == "p"

### General uptake constraints

Using the attribute `Reaction.bounds`

In [4]:
for reaction in model.exchanges:
    print(reaction.id, reaction.reaction)

Ca_tx  <=> CAII_e
Photon_tx  <=> Photon_e
H_tx PROTON_e --> 
Sucrose_tx  <=> SUCROSE_e
H2O_tx  <=> WATER_e
CO2_tx  <=> CARBON_DIOXIDE_e
O2_tx  <=> OXYGEN_MOLECULE_e
Pi_tx  <=> Pi_e
Mg_tx  <=> MGII_e
Nitrate_tx  <=> NITRATE_e
SO4_tx  <=> SULFATE_e
NH4_tx  <=> AMMONIUM_e
K_tx  <=> KI_e
GLC_tx  <=> GLC_e


In [5]:
# Skipped
# ATP bounds
model.reactions.get_by_id("ATPase_tx").bounds = (0, 1000)

# NTT is only active at night
model.reactions.get_by_id("ATP_ADP_Pi_pc").bounds = (0, 0)

In [6]:
# Defining autotrophic conditions
model.reactions.get_by_id("CO2_tx").bounds = (-1000, 1000)
model.reactions.get_by_id("H2O_tx").bounds = (-1000, 1000)
model.reactions.get_by_id("NH4_tx").bounds = (0, 0)
model.reactions.get_by_id("Pi_tx").bounds = (0, 1000)
model.reactions.get_by_id("SO4_tx").bounds = (0, 1000)
model.reactions.get_by_id("O2_tx").bounds = (-1000, 1000)

# Exported but not imported
model.reactions.get_by_id("Sucrose_tx").bounds = (-1000, 0)
model.reactions.get_by_id("GLC_tx").bounds = (-1000, 0)

### Creating and adding new reactions

Using `cobramod` to extend the model

In [7]:
# Malate/Pyruvate transporter
add_reactions(
    model=model,
    obj="PYR_MAL_pc, Pyruvate/Malate transporter | "
    + "PYRUVATE_p + MAL_c <-> PYRUVATE_c + MAL_p",
    directory=dir_data,
)

assert model.reactions.get_by_id("PYR_MAL_pc")

### Creating mesophyll and bundle sheath cells

We use copies of our model and add prefixes to them

In [8]:
c3_model = model.copy()
c4_leaf = model.copy()
c4_bundle = model.copy()

In [9]:
# For the mesophyll cell
for dictlist in [
    c4_leaf.reactions,
    c4_leaf.metabolites,
]:
    for item in dictlist:
        item.id = f"M_{item.id}"

# For the bundle sheath cell
for dictlist in [
    c4_bundle.reactions,
    c4_bundle.metabolites,
]:
    for item in dictlist:
        item.id = f"B_{item.id}"

c4_model = c4_leaf.merge(right=c4_bundle)

In [10]:
c4_bundle

Name,PlantCoreMetabolism_v1_3_0
Memory address,0x07fe8a68fa5e0
Number of metabolites,861
Number of reactions,893
Number of groups,208
Objective expression,1.0*B_Phloem_output_tx - 1.0*B_Phloem_output_tx_reverse_ef7a0
Compartments,"Mitochondrion, Cytoplasm, Biomass, Plastid, Vacuole, Peroxisome, Endoplasmic reticulum, Mitochondrion innermembrane interacting with cristal space, Mitochondrion innermembrane interacting with inter membrane space, Extracellular, Thylakoid, Mitochondrial intermembrane space"


In [11]:
c4_model

Name,PlantCoreMetabolism_v1_3_0
Memory address,0x07fe8a68c1d90
Number of metabolites,1629
Number of reactions,1786
Number of groups,208
Objective expression,1.0*M_Phloem_output_tx - 1.0*M_Phloem_output_tx_reverse_d57a8
Compartments,"Mitochondrion, Cytoplasm, Biomass, Plastid, Vacuole, Peroxisome, Endoplasmic reticulum, Mitochondrion innermembrane interacting with cristal space, Mitochondrion innermembrane interacting with inter membrane space, Extracellular, Thylakoid, Mitochondrial intermembrane space"


### Testing the new reactions

- Using the `assert` statement
- Confirm existence of reaction in cells

In [12]:
assert c4_model.reactions.get_by_id("M_CO2_tx")
assert c4_model.reactions.get_by_id("B_CO2_tx")
assert c4_model.reactions.get_by_id("M_ATPase_tx")
assert c4_model.reactions.get_by_id("B_ATPase_tx")

In [13]:
# Skipped
with open(file=Path.cwd().joinpath("model", "metabolites.txt")) as f:
    no_transport = [line[:-1].replace('"', "") for line in f.readlines()]

In [14]:
# Skipped
metabolites = []
for metabolite in c4_model.metabolites:
    if not metabolite.reactions:
        continue
    identifier = metabolite.id[2:]

    if identifier[: identifier.rfind("_")] in no_transport:
        continue
    elif f"MB_{identifier}" in [
        reaction.id for reaction in c4_model.reactions
    ]:
        continue
    elif "mc" not in metabolite.compartment and "c" in metabolite.compartment:
        metabolites.append(identifier)

### Adding transport reactions between the mesophyll and bundle sheath cells

- Filtering of metabolites (not shown) (unclear to me)
- Creating reactions
- Testing

In [15]:
for identifier in metabolites:
    add_reactions(
        model=c4_model,
        obj=f"MB_{identifier}, {identifier} bundle sheath/mesophyll cell "
        f"transporter | M_{identifier} <-> B_{identifier}",
        show_imbalance=False,
        directory=dir_data,
    )
    assert c4_model.reactions.get_by_id(f"MB_{identifier}")

/home/stefano/miniconda3/envs/test/lib/python3.9/site-packages/cobramod/core/creation.py:1129: UserWarning: Reaction "MB_PRO_c" is already present in the model. Skipping addition.
  warn(message=msg, category=UserWarning)
/home/stefano/miniconda3/envs/test/lib/python3.9/site-packages/cobramod/core/creation.py:1129: UserWarning: Reaction "MB_CIT_c" is already present in the model. Skipping addition.
  warn(message=msg, category=UserWarning)
/home/stefano/miniconda3/envs/test/lib/python3.9/site-packages/cobramod/core/creation.py:1129: UserWarning: Reaction "MB_2_KETOGLUTARATE_c" is already present in the model. Skipping addition.
  warn(message=msg, category=UserWarning)
/home/stefano/miniconda3/envs/test/lib/python3.9/site-packages/cobramod/core/creation.py:1129: UserWarning: Reaction "MB_LEU_c" is already present in the model. Skipping addition.
  warn(message=msg, category=UserWarning)
/home/stefano/miniconda3/envs/test/lib/python3.9/site-packages/cobramod/core/creation.py:1129: UserW

In [16]:
!cat model/metabolites.txt

NITRATE
NITRITE
OXYGEN_MOLECULE
aHS
SULFATE
WATER
FRUCTOSE_16_DIPHOSPHATE
DPG
PROTON
ACETALD
ACET
5_10_METHENYL_THF
5_METHYL_THF
HOMO_CYS
ADENOSYL_HOMO_CYS
ACETYLSERINE
THF
ADENOSINE
MALTOSE
CO_A
L_GLUTAMATE_5_P
aL_GLUTAMATE_5_P_c ACETYL_COA
CELLULOSE
L_GLUTAMATE_GAMMA_SEMIALDEHYDE
S_ADENOSYLMETHIONINE
PPI
L_DELTA1_PYRROLINE_5_CARBOXYLATE
AMMONIUM
Pi
CARBON_DIOXIDE
OXALACETIC_ACID
HCO3
UTP
aUTP
UDP
aUDP
UDP_GLUCOSE
ATP
aATP
ADP
aADP
AMP
IMP
aIMP
XANTHOSINE_5_PHOSPHATE
aXANTHOSINE_5_PHOSPHATE
GTP
aGTP
GDP
aGDP
GMP
aGMP
bGMP
GDP
aGDP
CDP
DUMP
DTMP
aDTDP
GTP
DTTP aDTTP
NAD
NADH
NADP
NADPH


### C4-specific constrains

- No CO2 uptake in bundle sheath cells
- Block rubisco activity in the mesophyll
- No oxygenation activity of bundle sheath rubisco (Stefano, is this set?)
- Enforce NADP-ME decarboxylation pathways (e.g. in maize)

Examples:

- Notes

- CONSTRAINT: No CO2 uptake in bundle sheath cells due to suberin layer in cell membranes
- Force C4 cycle: Block Rubisco carboxylase/oxygenase in Mesophyll
- Force NADP-ME decarboxylation pathway: Block all other decarboxylation reactions except NADP_ME in the plastid/ make alternative decarboxylation routes irreversible

In [17]:
c4_model.reactions.get_by_id("B_CO2_tx").bounds = (0, 0)
# Carboxylase
c4_model.reactions.get_by_id(
    "M_RIBULOSE_BISPHOSPHATE_CARBOXYLASE_RXN_p"
).bounds = (0, 0)
# Oxygenase
c4_model.reactions.get_by_id("M_RXN_961_p").bounds = (0, 0)

In [18]:
# Force NADP-ME decarboxylation pathway:
# Block all other decarboxylation reactions except NADP_ME in the plastid
c4_model.reactions.get_by_id("B_PEPCARBOXYKIN_RXN_c").bounds = (0, 0)
c4_model.reactions.get_by_id(
    "B_1_PERIOD_1_PERIOD_1_PERIOD_39_RXN_m"
).bounds = (0, 0)
c4_model.reactions.get_by_id("B_MALIC_NADP_RXN_c").bounds = (0, 0)


In [19]:
# Force NADP-ME decarboxylation pathways:
# make alternative decarboxylation routes irreversible
c4_model.reactions.get_by_id("B_CARBAMATE_KINASE_RXN_p").bounds = (
    0,
    1000,
)
c4_model.reactions.get_by_id("M_CARBAMATE_KINASE_RXN_p").bounds = (
    0,
    1000,
)
c4_model.reactions.get_by_id("B_ISOCITDEH_RXN_m").bounds = (0, 1000)
c4_model.reactions.get_by_id("M_ISOCITDEH_RXN_m").bounds = (0, 1000)
c4_model.reactions.get_by_id("B_ISOCITDEH_RXN_c").bounds = (0, 1000)
c4_model.reactions.get_by_id("M_ISOCITDEH_RXN_c").bounds = (0, 1000)
c4_model.reactions.get_by_id("B_ISOCITRATE_DEHYDROGENASE_NAD_RXN_m").bounds = (
    0,
    1000,
)
c4_model.reactions.get_by_id("M_ISOCITRATE_DEHYDROGENASE_NAD_RXN_m").bounds = (
    0,
    1000,
)

In [20]:
# skipped
# Fix malate transport
c4_model.reactions.get_by_id("B_OAA_MAL_pc").bounds = (0, 1000)

c4_model.reactions.get_by_id("B_PYRUVATE_pc").bounds = (0, 0)

c4_model.reactions.get_by_id(
    "M_PYRUVATEORTHOPHOSPHATE_DIKINASE_RXN_c"
).bounds = (0, 0)

In [21]:
# Skipped
# NGAM for C3
const = c3_model.problem.Constraint(
    (0.0049 * c3_model.reactions.Photon_tx.flux_expression + 2.7852)
    - c3_model.reactions.ATPase_tx.flux_expression,
    lb=0,
    ub=0,
)
c3_model.add_cons_vars(const)

# ATP/NADPH 3:1 constraints
const = c3_model.problem.Constraint(
    c3_model.reactions.ATPase_tx.flux_expression
    - 3
    * (
        c3_model.reactions.NADPHoxc_tx.flux_expression
        + c3_model.reactions.NADPHoxp_tx.flux_expression
        + c3_model.reactions.NADPHoxm_tx.flux_expression
    ),
    lb=0,
    ub=0,
)
c3_model.add_cons_vars(const)

# Rubisco
# ATP/NADPH 3:1 constraints
const = c3_model.problem.Constraint(
    3 * c3_model.reactions.RXN_961_p.flux_expression -
    c3_model.reactions.RIBULOSE_BISPHOSPHATE_CARBOXYLASE_RXN_p.flux_expression, 
    lb=0,
    ub=0,
)
c3_model.add_cons_vars(const)

### Using more complex constrains

- Using `Constraint` objects
- Define expression and their bounds
- Can combine multiple reactions

Example: Sum of photon uptake in mesophyll and bundle sheath
cells cannot surpass 200

In [22]:
# Skipped
# Constrains for light dependent maintenance costs
atp_b = c4_model.reactions.get_by_id("B_ATPase_tx")
photon_b = c4_model.reactions.get_by_id("B_Photon_tx")
atp_m = c4_model.reactions.get_by_id("M_ATPase_tx")
photon_m = c4_model.reactions.get_by_id("M_Photon_tx")

const_b = c4_model.problem.Constraint(
    (0.0049 * photon_b.flux_expression + 2.7852) - atp_b.flux_expression,
    lb=0,
    ub=0,
)

const_m = c4_model.problem.Constraint(
    (0.0049 * photon_m.flux_expression + 2.7852) - atp_m.flux_expression,
    lb=0,
    ub=0,
)
c4_model.add_cons_vars([const_b, const_m])

In [23]:
# Light Uptake constrain
B_photon = c4_model.reactions.get_by_id("B_Photon_tx").flux_expression
M_photon = c4_model.reactions.get_by_id("M_Photon_tx").flux_expression

# CONSTRAINT: Total Photon uptake limited to "light" µE
photon_sum = c4_model.problem.Constraint(
    B_photon + M_photon, lb=0, ub=200
)

# CONSTRAINT: Total Photon uptake by bundle sheath must be less or
# equal than in mesophyll
photon_ratio = c4_model.problem.Constraint(
    M_photon - B_photon, lb=0, ub=200
)

c4_model.add_cons_vars([photon_sum, photon_ratio])

In [24]:
# Skipped
# CONSTRAINT : Total N uptake must not surpass defined upper bound
bs_nitrate = c4_model.reactions.get_by_id("B_Nitrate_tx").flux_expression
m_nitrate = c4_model.reactions.get_by_id("M_Nitrate_tx").flux_expression

nitrate_ratio = c4_model.problem.Constraint(
    bs_nitrate + m_nitrate, lb=0, ub=1000
)

c4_model.add_cons_vars([nitrate_ratio])

In [25]:
# Skipped
# ATP/NADPH 3:1 constraints
for cell in ["B_", "M_"]:
    const = c4_model.problem.Constraint(
        c4_model.reactions.get_by_id(f"{cell}ATPase_tx").flux_expression
        - 3
        * (
            c4_model.reactions.get_by_id(f"{cell}NADPHoxc_tx").flux_expression
            + c4_model.reactions.get_by_id(
                f"{cell}NADPHoxp_tx"
            ).flux_expression
            + c4_model.reactions.get_by_id(
                f"{cell}NADPHoxm_tx"
            ).flux_expression
        ),
        lb=0,
        ub=0,
    )
    c4_model.add_cons_vars(const)


### Simulating the model by optimising biomass production

$\rightarrow$ new biomass reaction (there should not be a "new" biomass)

Solution example:

In [26]:
c4_model.objective = "B_AraCore_Biomass_tx"
c4_model.optimize()

,fluxes,reduced_costs
M_PRO_PROTON_vc,0.000000e+00,0.000000e+00
M_Ca_tx,0.000000e+00,-0.000000e+00
M_H2O_xc,-4.458611e-01,-0.000000e+00
M_sCIT_biomass,0.000000e+00,-0.000000e+00
M_ACETYLGLUTKIN_RXN_p,0.000000e+00,-1.734723e-18
...,...,...
MB_SINAPATE_c,-4.805257e-16,0.000000e+00
MB_SYRINGALDEHYDE_c,4.805257e-16,-0.000000e+00
MB_SYRINGIC_ACID_c,0.000000e+00,-8.326673e-17
MB_VANILLIC_ACID_c,0.000000e+00,-8.196568e-17


### Obtaining solutions

Saving solutions for the C4 and C3 model to comparison

In [27]:
c4_model.objective = "B_AraCore_Biomass_tx"
c4_solution = c4_model.optimize()

In [28]:
c3_model.reactions.get_by_id("Photon_tx").bounds = (0, 200)
c3_model.objective = "AraCore_Biomass_tx"
c3_solution = c3_model.optimize()

### C4 mechanismus 

<img src="img/C4_model.png" alt="drawing" width="680"/>
<font size="5">PEPC: PEP Carboxylase; MAL-DEH : Malate Dehydrogenase; NADP-ME: NADP Malic enzyme; PPDK: Pyruvate, phosphate dikinase.</font>
 


### Comparison

#### Malate Dehydrogenase

In [29]:
for reaction in c4_model.reactions.query("MALATE_DEH_RXN_[c,p]"):
    print(reaction.id,  c4_solution.fluxes[reaction.id])

M_MALATE_DEH_RXN_c 0.42254251702609696
M_MALATE_DEH_RXN_p 0.0
B_MALATE_DEH_RXN_c -0.6833695023140559
B_MALATE_DEH_RXN_p 0.0


In [30]:
for reaction in c3_model.reactions.query("MALATE_DEH_RXN_[c,p]"):
    print(reaction.id,  c3_solution.fluxes[reaction.id])

MALATE_DEH_RXN_c 1.0948460727088143
MALATE_DEH_RXN_p 4.630936553466511


#### RuBisco reactions

In [31]:
for reaction in c4_model.reactions.query("RIBULOSE_BISPHOSPHATE_CARBOXYLASE_RXN"):
    print(reaction.id,  c4_solution.fluxes[reaction.id])
for reaction in c4_model.reactions.query("RXN_961"):
    print(reaction.id,  c4_solution.fluxes[reaction.id])

M_RIBULOSE_BISPHOSPHATE_CARBOXYLASE_RXN_p 0.0
B_RIBULOSE_BISPHOSPHATE_CARBOXYLASE_RXN_p 16.443645880987933
M_RXN_961_p 0.0
B_RXN_961_p 0.0


In [32]:
for reaction in c3_model.reactions.query("RIBULOSE_BISPHOSPHATE_CARBOXYLASE_RXN"):
    print(reaction.id,  c3_solution.fluxes[reaction.id])
for reaction in c3_model.reactions.query("RXN_961"):
    print(reaction.id,  c3_solution.fluxes[reaction.id])

RIBULOSE_BISPHOSPHATE_CARBOXYLASE_RXN_p 14.21464721787285
RXN_961_p 4.73821573929095


#### Nitrate intake

In [33]:
for reaction in c4_model.reactions.query("Nitrate_tx"):
    print(reaction.id,  c4_solution.fluxes[reaction.id])

M_Nitrate_tx 0.0
B_Nitrate_tx 1.3301677953996625


In [34]:
for reaction in c3_model.reactions.query("Nitrate_tx"):
    print(reaction.id,  c3_solution.fluxes[reaction.id])

Nitrate_tx 1.0823432489141842


#### CO2 intake and O2 output 

In [35]:
for reaction in c4_model.reactions.query("O2_tx"):
    print(reaction.id,  c4_solution.fluxes[reaction.id])

M_CO2_tx 15.161925330403678
M_O2_tx -11.014842693408811
B_CO2_tx 0.0
B_O2_tx -7.683563978289961


In [36]:
for reaction in c3_model.reactions.query("O2_tx"):
    print(reaction.id,  c3_solution.fluxes[reaction.id])

CO2_tx 12.337095799987418
O2_tx -15.214692685057068


### Questions?

### Thank you very much!!